In [72]:
import numpy as np
import pandas as pd
from scipy import misc

import os
import shutil
from tqdm import tqdm 

from PIL import Image

In [73]:
# the folder for images
dir_train = 'E:/Jupyter NB/10 class Classification/Images'

image_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test'

# This cell is for testing

In [74]:
for root, dirs, files in os.walk(dir_train):
    print('---root---')
    print(root)
    print('---dirs---')
    print(dirs)
    print('---files---')
    print(files)
    print('---------------------')



subdirs = list(os.walk(dir_train))[1:]

for dir_path, _, files in tqdm(subdirs):
    
    dir_name = dir_path.split('\\')[-1]
    print(dir_name)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]

---root---
E:/Jupyter NB/10 class Classification/Images
---dirs---
['01.backpack', '02.bathtub', '03.billiards', '04.binoculars', '05.bonsai-101', '06.boxing-glove', '07.breadmaker', '08.chess-board', '09.cockroach', '10.coin']
---files---
[]
---------------------
---root---
E:/Jupyter NB/10 class Classification/Images\01.backpack
---dirs---
[]
---files---
['003_0001.jpg', '003_0002.jpg', '003_0003.jpg', '003_0004.jpg', '003_0005.jpg', '003_0006.jpg', '003_0007.jpg', '003_0008.jpg', '003_0009.jpg', '003_0010.jpg', '003_0011.jpg', '003_0012.jpg', '003_0013.jpg', '003_0014.jpg', '003_0015.jpg', '003_0016.jpg', '003_0017.jpg', '003_0018.jpg', '003_0019.jpg', '003_0020.jpg', '003_0021.jpg', '003_0022.jpg', '003_0023.jpg', '003_0024.jpg', '003_0025.jpg', '003_0026.jpg', '003_0027.jpg', '003_0028.jpg', '003_0029.jpg', '003_0030.jpg', '003_0031.jpg', '003_0032.jpg', '003_0033.jpg', '003_0034.jpg', '003_0035.jpg', '003_0036.jpg', '003_0037.jpg', '003_0038.jpg', '003_0039.jpg', '003_0040.jpg', 

In [75]:
subdirs = list(os.walk(dir_train))[1:]

# collect train metadata
train_metadata = []

for dir_path, _, files in tqdm(subdirs):
    
    dir_name = dir_path.split('\\')[-1]
    
    for file_name in files:
        if not file_name.startswith('.'):
            # read image
            temp = misc.imread(os.path.join(dir_path, file_name)) 
            # collect image metadata
            image_metadata = []
            image_metadata.extend([dir_name, file_name])
            image_metadata.extend( 
                list(temp.shape) if len(temp.shape) == 3 
                else [temp.shape[0], temp.shape[1], 1]
            )
            image_metadata.extend([temp.nbytes, temp.dtype])
            # append image metadata to list
            train_metadata.append(image_metadata)

#print(train_metadata)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\aRnob\miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.65it/s]


In [76]:
M = pd.DataFrame(train_metadata)
M.columns = ['directory', 'img_name', 'height', 'width', 'channels', 'byte_size', 'bit_depth']

M['category_name'] = M.directory.apply(lambda x: x.split('.')[-1].lower())
M['img_extension'] = M.img_name.apply(lambda x: x.split('.')[-1])
M['category_number'] = M.directory.apply(lambda x: int(x.split('.')[0]))

In [77]:
# number of grayscale images
(M.channels != 3).sum()

8

In [78]:
# number of categories
M.category_name.nunique()

10

In [79]:
# class number -> class name
decode = {n: i for i, n in M.groupby('category_name').category_number.first().iteritems()}
decode

np.save('decode.npy', decode)

In [80]:
# 20 images per class
V = M.groupby('category_name', group_keys=False).apply(lambda x: x.sample(n=20, replace=False))
V.sort_index(inplace=True)
M.drop(V.index, axis=0, inplace=True)

# 20 images per class for test
Test = M.groupby('category_name', group_keys=False).apply(lambda x: x.sample(n=20, replace=False))
Test.sort_index(inplace=True)
M.drop(Test.index, axis=0, inplace=True)

In [81]:
# train data
len(M)

1233

In [82]:
# validation data
len(V)

200

In [83]:
#Test data
len(Test)

200

In [84]:
M.to_csv('train_metadata.csv', index=False)
V.to_csv('val_metadata.csv', index=False)
Test.to_csv('test_data.csv', index=False)

In [85]:
t = pd.read_csv('E:/Jupyter NB/10 class Classification/train_metadata.csv')
v = pd.read_csv('E:/Jupyter NB/10 class Classification/val_metadata.csv')
test = pd.read_csv('E:/Jupyter NB/10 class Classification/test_data.csv')

In [86]:
os.mkdir(image_dir + '/train')
for i in range(1, 10 + 1):
    os.mkdir(image_dir + '/train/' + str(i))

In [87]:
os.mkdir(image_dir + '/val')
for i in range(1, 10 + 1):
    os.mkdir(image_dir + '/val/' + str(i))

In [88]:
os.mkdir(image_dir + '/test')
for i in range(1, 10 + 1):
    os.mkdir(image_dir + '/test/' + str(i))

# Train images

In [89]:
train_size = len(t)
train_size

1233

In [90]:
# RGB images
for i, row in tqdm(t.loc[t.channels == 3].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # save
    save_path = os.path.join(image_dir, 'train', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

1229it [00:07, 158.51it/s]


In [92]:
# grayscale images
for i, row in tqdm(t.loc[t.channels == 1].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # convert to RGB
    array = np.asarray(image, dtype='uint8')
    array = np.stack([array, array, array], axis=2)
    image = Image.fromarray(array)
    
    # save
    save_path = os.path.join(image_dir, 'train', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

4it [00:00, 52.06it/s]


# Validation images

In [96]:
val_size = len(v)
val_size

200

In [94]:
# RGB images
for i, row in tqdm(v.loc[v.channels == 3].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # save
    save_path = os.path.join(image_dir, 'val', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

196it [00:01, 140.69it/s]


In [95]:
# grayscale images
for i, row in tqdm(v.loc[v.channels == 1].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # convert to RGB
    array = np.asarray(image, dtype='uint8')
    array = np.stack([array, array, array], axis=2)
    image = Image.fromarray(array)
    
    # save
    save_path = os.path.join(image_dir, 'val', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

4it [00:01,  2.10it/s]


# Test images

In [97]:
test_size = len(test)
test_size

200

In [98]:
# RGB images
for i, row in tqdm(test.loc[test.channels == 3].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # save
    save_path = os.path.join(image_dir, 'test', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

200it [00:01, 152.59it/s]


In [99]:
# grayscale images
for i, row in tqdm(test.loc[test.channels == 1].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # convert to RGB
    array = np.asarray(image, dtype='uint8')
    array = np.stack([array, array, array], axis=2)
    image = Image.fromarray(array)
    
    # save
    save_path = os.path.join(image_dir, 'test', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

0it [00:00, ?it/s]
